In [64]:
import json

# Load secrets and prepare gdrive


In [65]:
import os
from google.colab import userdata

os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')
os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')
os.environ['WANDB_API_KEY'] = userdata.get('WANDB_API_KEY')

In [66]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [67]:
import wandb
wandb.login(key=os.environ['WANDB_API_KEY'])

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Check out repo


In [68]:
!rm -r any2json
!git clone --filter=blob:none  https://github.com/btseytlin/any2json.git
!cd any2json && git fetch && git reset --hard origin/main

Cloning into 'any2json'...
remote: Enumerating objects: 520, done.
remote: Counting objects: 100% (158/158), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 520 (delta 108), reused 100 (delta 50), pack-reused 362 (from 1)
Receiving objects: 100% (520/520), 47.42 KiB | 408.00 KiB/s, done.
Resolving deltas: 100% (289/289), done.
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 51 (delta 1), reused 5 (delta 0), pack-reused 13 (from 1)
Receiving objects: 100% (51/51), 259.98 KiB | 1.31 MiB/s, done.
Resolving deltas: 100% (1/1), done.
HEAD is now at 5747717 Vibin


In [69]:
!cd any2json && uv -q pip install -e .

# Download dataset

In [70]:
from datasets import load_dataset

DATASET_ID = 'btseytlin/any2json'

dataset = load_dataset(DATASET_ID)

print("Train size", len(dataset['train']))
print("Test size", len(dataset['test']))
dataset['train']

Train size 183663
Test size 4959


Dataset({
    features: ['input_data', 'schema', 'output', 'meta'],
    num_rows: 183663
})

# Run Training

In [71]:
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
!python any2json/any2json/training/train.py train --bf16 --per-device-train-batch-size 1 --gradient-accumulation-steps 10 --debug-limit 100 --num-train-epochs=10  --max-source-length=2048 --max-target-length=2048 --logging-steps 50 --eval-steps 50 --save-steps 100000

2025-08-11 15:48:04.530067: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-11 15:48:04.548930: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754927284.570480   10599 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754927284.577072   10599 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754927284.594391   10599 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

# Upload results

In [72]:
# import shutil
# import gzip
# from datetime import datetime

# dt = datetime.now().isoformat()

# # upload benchmark results to gdrive

# out_path = f"/content/drive/MyDrive/any2json/benchmarks/"
# os.makedirs(out_path, exist_ok=True)

# zip_path = f"/content/benchmark_results_{dt}"
# shutil.make_archive(zip_path, "zip", "/content/benchmark_results")

# shutil.copy(f"{zip_path}.zip", out_path)

# Shutdown

In [73]:
# # shutdown instance
# from google.colab import runtime
# runtime.unassign()